In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#tf.keras.utils.img_to_array
#tf.keras.utils.load_img
from tensorflow.keras.utils import img_to_array, load_img

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-06-09 21:40:34.469496: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
path = 'src/images/images/'
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

train_gen = train_datagen.flow_from_directory(path + 'train',
                                              target_size=(48, 48),
                                              color_mode='grayscale',
                                              batch_size= 128,
                                              class_mode= 'categorical',
                                              shuffle= True)
val_gen = val_datagen.flow_from_directory(path+ 'validation',
                                          target_size= (48, 48),
                                          color_mode= 'grayscale',
                                          batch_size= 128,
                                          class_mode= 'categorical',
                                          shuffle= False)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding= 'same', input_shape= (48, 48, 1), activation= 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.15),

    tf.keras.layers.Conv2D(128, (3, 3), activation= 'relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.15),

    tf.keras.layers.Conv2D(128, (3, 3), activation= 'relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Conv2D(256, (3, 3), activation= 'relu', padding= 'same'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dense(256, activation= 'relu'),
    tf.keras.layers.Dense(7, activation= 'softmax')
])
model.compile(optimizer= tf.keras.optimizers.Adam(), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [31]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('3052023-2302.h5', monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [32]:
history= model.fit(train_gen,
                             steps_per_epoch= 225,
                             epochs= 200,
                             validation_data= val_gen,
                             validation_steps= 100,
                   callbacks= callbacks_list)

Epoch 1/200
225/225 [==============================] - ETA: 0s - loss: 2.1899 - accuracy: 0.2516WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 100 batches). You may need to use the repeat() function when building your dataset.

Epoch 1: accuracy improved from -inf to 0.25163, saving model to 3052023-2302.h5
225/225 [==============================] - 60s 263ms/step - loss: 2.1899 - accuracy: 0.2516 - val_loss: 1.7074 - val_accuracy: 0.3368
Epoch 2/200
225/225 [==============================] - ETA: 0s - loss: 1.6266 - accuracy: 0.3625
Epoch 2: accuracy improved from 0.25163 to 0.36246, saving model to 3052023-2302.h5
225/225 [==============================] - 54s 240ms/step - loss: 1.6266 - accuracy: 0.3625
Epoch 3/200
225/225 [==============================] - ETA: 0s - loss: 1.4939 - accuracy: 0.4187
Epoch 3: accuracy improved from 0.36246 to 0.41871,

In [35]:
# serialize model structure to JSON
model_json = model.to_json()
with open("3052023-2302", "w") as json_file:
    json_file.write(model_json)

In [34]:
model.save_weights('./model')
model.save('./model')

INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: ./model/assets


In [47]:
#model.save('latestmodel.h5')

In [4]:
from keras.models import load_model
model = load_model('FINAL 97%/3052023-2302.h5')

In [16]:
inputgajelas= 'src/images/validation/happy/366.jpg'
img = load_img(inputgajelas, target_size= (48, 48), grayscale= True)

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(score)

1/1 [==============================] - 0s 18ms/step
tf.Tensor(
[0.11530168 0.11493786 0.11493971 0.30915797 0.11577818 0.11494576
 0.11493883], shape=(7,), dtype=float32)
